# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [2]:
import sqlite3
import pandas as pd

In [3]:
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [4]:
cur.execute("""SELECT customerNumber, contactLastName, contactFirstName
               FROM customers
               WHERE customerNumber = (SELECT customerNumber
                                       FROM orders
                                       WHERE orderDate = '2003-01-31')
               ;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [10]:
cur.execute("""SELECT productName, 
                      COUNT(distinct orderNumber) as numberOrders, 
                      SUM(quantityOrdered) as totalUnitsSold
               FROM products
               INNER JOIN orderdetails
               USING(productCode)
               GROUP BY productCode
               ORDER BY totalUnitsSold DESC
               ;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(f'Number of distinct items sold: {len(df)}')

Number of distinct items sold: 109


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

In [11]:
cur.execute("""SELECT productName, 
                      COUNT(distinct customerNumber) as totalnumberofpeople
               FROM products
               INNER JOIN orderdetails
               USING(productCode)
               INNER JOIN orders
               USING(orderNumber)
               GROUP BY productCode
               ORDER BY totalnumberofpeople DESC
               ;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(f'Number of distinct items sold: {len(df)}')

Number of distinct items sold: 109


## Select the Employee Number, Office Code, City (of the office), and Name (First and Last) of those Employees who Sold Products that Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [20]:
cur.execute("""SELECT productName
               FROM products
               INNER JOIN orderdetails
               USING(productCode)
               INNER JOIN orders
               USING(orderNumber)
               GROUP BY productCode
               HAVING COUNT(distinct customerNumber) <= 20
               ;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

,productName
0,Collectable Wooden Train
1,1976 Ford Gran Torino
2,1957 Ford Thunderbird
3,1970 Chevy Chevelle SS 454
4,1970 Dodge Coronet
5,1949 Jaguar XK 120
6,1958 Chevy Corvette Limited Edition
7,1952 Citroen-15CV
8,1969 Chevrolet Camaro Z28
9,2002 Chevy Corvette


In [14]:
cur.execute("""SELECT employeeNumber,
                      officeCode,
                      offices.city,
                      firstName,
                      lastName
               FROM employees
               INNER JOIN offices
               USING(officeCode)
               JOIN customers on employees.employeeNumber=customers.salesRepEmployeeNumber 
               WHERE salesRepEmployeeNumber = (SELECT productName
                                               FROM products
                                               INNER JOIN orderdetails
                                               USING(productCode)
                                               INNER JOIN orders
                                               USING(orderNumber)
                                               GROUP BY productCode
                                               HAVING COUNT(distinct customerNumber) <= 20)
               ;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df

OperationalError: near "INNNER": syntax error

## Select the Employee Number, Name (First and Last) and Number of Customers of Employees Who's Customers Have an Average Credit Limit of Over 15K

In [ ]:
#Your code here

## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!